# Practical 9: Web Mining

### In this practical
1. [Loading data](#load)
2. [Cleaning web logs](#prep)
3. [Applying data mining methods to cleaned data](#application)

---
**Written by Hendi Lie (h2.lie@qut.edu.au) and Richi Nayak (r.nayak@qut.edu.au). All rights reserved.**

This practical note introduces you to the web data processing for performing mining in Python. The input file for this exercise is `datasets/wdata.txt` that contains web log data in text format. You will learn to clean and perform user session identification in preparation of applying one of the data mining techniques you have learned so far in the previous practicals.

## 1. Loading Data

Web mining is a branch of data mining that concentrates on mining useful information from the web. The significant tasks of this web mining include resource finding, information selection, preprocessing, generalisation and analysis.

There are many types of web mining, including web usage mining, web structure mining and web content mining. Web usage mining, in particular, have allowed organisation to analyse user usage patterns, resulting in great insights for organisations to improve site design, identify potential customers and improve search results.

In general, log analysis is include in web usage mining process. It takes raw web data and process them in order to extract statistical information, such as:
* Key statistical figures (number of visitors, average number of hits, view time, etc)
* Diagnostic statistics (server reports and page not found errors)
* Server statistics (top pages visited, entry/exit pages)
* Referrer statistics (top referrering sites, search engine, key words)
* User demographics, client statistics and so on.

Web usage mining commonly uses web log data, which contain raw information related to pages served and recorded by the web server. This raw information is not sufficient and is not accurate to infer the behavior of the user. Thus, we need to perform preprocessing to extract meaningful information.

In this practical, we will be using `wdata.txt` log dataset. Load them using files (not pandas) as follows:

In [1]:
# load logs from wdata
wdata = open('datasets/wdata.txt', 'r').readlines()

# print the first 3 lines
print('\n'.join(wdata[:3]))

web_logs

j2439.inktomisearch.com - - [18/Apr/2005:21:16:54 +1000] "GET /robots.txt HTTP/1.0" 404 204 "-" "Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)"

lj2559.inktomisearch.com - - [18/Apr/2005:21:16:55 +1000] "GET /code/Global/code/menu.html HTTP/1.0" 200 6092 "-" "Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)"



From the first couple of lines from this file, we could see the structure of this dataset. Its contents are:

1. **Host**:  The first part of the web log is called host. This is either ip address (202.183.101.13) or  the  host  name (lj2439.inktomisearch.com)  of  the  remote  user  requesting  the  page. For  performance  reasons, many  web  servers  are  configured  to  publish  their  IP  address instead of host name, but in this dataset, the host contains host names.
2. **Identd result**: The  dash  (-)  next  to  the  host  name  represents  the  logging  response  returned  by  the  remote  user’s  Identd  result. Almost no  web servers uses this; in most web log this field is always just a dash (-).    
3. **Authuser**: The next part of the log displays the authentication code of the user if there exists any for that particular web site or else just dash (-) is displayed. 
4. **Date and time**: Next to come in the   row   is   the   date   and   time   inside   the   square   brackets [18/Apr/2005:21:16:54  +1000].  It is in the day/month/year  format.  The  time  is  followed  by the date is displayed in 24 hours format with time zone offset at the end. The time-zone offset corresponds to Universal Time/Greenwich Mean Time.
5. **Request**: This  is  the  request  sent  by  the  user  enclosed  in  double  quotes.  Normally  it  looks  something  like  “GET  /robots.txt  HTTP/1.0".  In  this  part  the  **GET**  represents  request method, **/robots.txt** is the path of requested web page and **HTTP/1.0** being the request protocol.
6. **Status code**: This  is  a  3-digit  code  returned  by  the  server  indicating  the  status  of  the  request  to  server. For example the code 200 stands for successful completion and 404 stands for unsuccessful completion or if the page could not be found.
7. **Bytes sent**: This represents the amount of the data delivered from the server excluding the header line.  
 
The extended version of this log format is called combined log format, with addition of two more fields.
8. **Agent**: The  user  agent  reported  by  the  requesting  user's  browser.  Typically,  this  is  a  string  describing the type and version of browser software being used.
9. **Referrer**: This is the referencing page of the user, in this case the referring URL is: http://help.yahoo.com/help/us/ysearch/slurp.  

Once we know the columns available in this dataset, you could reload the dataset using the `.read_csv` function. Each field is separated by spaces, thus we should specify its `sep` or separator as ' ' or space. We have also added `names` variable into `names` parameter of the read function to allow pandas set column names during read process.

In [2]:
import pandas as pd

# set names of pandas dataframe
names=['Host', 'Identd', 'Authuser', 'Date and time', 'Timezone', 'Request',
       'Status code', 'Bytes Sent', 'Referrer', 'Agent']
# read the dataframe
df = pd.read_csv('datasets/wdata.txt', sep=' ', names=names, header=None)

In [3]:
# preview
df.head()

,Host,Identd,Authuser,Date and time,Timezone,Request,Status code,Bytes Sent,Referrer,Agent
0,web_logs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,j2439.inktomisearch.com,-,-,[18/Apr/2005:21:16:54,+1000],GET /robots.txt HTTP/1.0,404.0,204,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...
2,lj2559.inktomisearch.com,-,-,[18/Apr/2005:21:16:55,+1000],GET /code/Global/code/menu.html HTTP/1.0,200.0,6092,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...
3,c210-49-32-6.rochd2.qld.optusnet.com.au,-,-,[18/Apr/2005:21:25:07,+1000],GET / HTTP/1.1,200.0,7138,http://www.google.com.au/search?hl=en&q=snap+p...,Mozilla/5.0 (Macintosh; U; PPC Mac OS X; en-us...
4,c210-49-32-6.rochd2.qld.optusnet.com.au,-,-,[18/Apr/2005:21:25:07,+1000],GET /images/index3_01.gif HTTP/1.1,200.0,382,http://www.copyspecialists.com.au/,Mozilla/5.0 (Macintosh; U; PPC Mac OS X; en-us...


The first line/row of this data is not part of the web logs, thus we should drop it.

In [4]:
df.drop(0, inplace=True)  # drop the row with index 0, on axis 0 (row-wise)

Notice that the `Request` column still has both HTTP request method and protocol in it. The code below will separate these information from the column into their respective columns.

In [5]:
def extract_method_and_protocol(row):
    # function to extract HTTP request method and protocol from a request string
    request_splits = row['Request'].split()  # split request string by space
    row['Method'] = request_splits[0]
    row['Protocol'] = request_splits[-1]
    row['Request'] = ' '.join(request_splits[1:-1])  # stitch remaining request string back
    return row

df = df.apply(extract_method_and_protocol, axis=1)

# show the result
df.head()

,Host,Identd,Authuser,Date and time,Timezone,Request,Status code,Bytes Sent,Referrer,Agent,Method,Protocol
1,j2439.inktomisearch.com,-,-,[18/Apr/2005:21:16:54,+1000],/robots.txt,404.0,204,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,GET,HTTP/1.0
2,lj2559.inktomisearch.com,-,-,[18/Apr/2005:21:16:55,+1000],/code/Global/code/menu.html,200.0,6092,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,GET,HTTP/1.0
3,c210-49-32-6.rochd2.qld.optusnet.com.au,-,-,[18/Apr/2005:21:25:07,+1000],/,200.0,7138,http://www.google.com.au/search?hl=en&q=snap+p...,Mozilla/5.0 (Macintosh; U; PPC Mac OS X; en-us...,GET,HTTP/1.1
4,c210-49-32-6.rochd2.qld.optusnet.com.au,-,-,[18/Apr/2005:21:25:07,+1000],/images/index3_01.gif,200.0,382,http://www.copyspecialists.com.au/,Mozilla/5.0 (Macintosh; U; PPC Mac OS X; en-us...,GET,HTTP/1.1
5,c210-49-32-6.rochd2.qld.optusnet.com.au,-,-,[18/Apr/2005:21:25:07,+1000],/images/index3_02.gif,200.0,1284,http://www.copyspecialists.com.au/,Mozilla/5.0 (Macintosh; U; PPC Mac OS X; en-us...,GET,HTTP/1.1


After the loading process is completed, we should explore this dataset. Run the following code cells for exploration.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51327 entries, 1 to 51327
Data columns (total 12 columns):
Host             51327 non-null object
Identd           51327 non-null object
Authuser         51327 non-null object
Date and time    51327 non-null object
Timezone         51327 non-null object
Request          51327 non-null object
Status code      51327 non-null float64
Bytes Sent       51327 non-null object
Referrer         51327 non-null object
Agent            51327 non-null object
Method           51327 non-null object
Protocol         51327 non-null object
dtypes: float64(1), object(11)
memory usage: 5.1+ MB


There are **51327** columns in this dataset, with most columns being object/string type.

## 2. Cleaning web logs

This section discusses techniques and steps taken to process the imported, raw web log data.

### 2.1. Removing useless requests

In this  process, we are removing requests relating to non-analysed resources such as extraneous references to 
embedded objects, graphics, sound files, and removing references due to spider navigations. In addition, we also remove all unsuccessful request (i.e. status code other than 200). A significant reason for cleaning these logs is to reduce storage space and facilitate the upcoming data mining tasks effectively. This list might change when planning for specific analysis. For example, when looking to analyse the performance web cache  application, there is a need for having image and graphic files in the dataset. 

Use the following code cell to remove image requests from the log such as `.gif`, `.jpg` and `.jpeg` and removing logs with status code other than 200.

In [7]:
# correct the incorrect dataframe types
df['Status code'] = df['Status code'].astype(int)  # set status code to int
df['Datetime'] = pd.to_datetime(df['Date and time'], format='[%d/%b/%Y:%H:%M:%S')  # set date time to pandas datatime obj
df = df.drop(['Date and time'], axis=1)

# create a mask to filter all images
mask = (df['Request'].str.endswith('.gif') | df['Request'].str.endswith('.jpg') | df['Request'].str.endswith('.jpeg'))
print("# Rows before:", len(df))

# invert the mask, only keep records without .gif, .jpg and .jpeg in the request column
df2 = df[~mask]

print("After images removal", len(df2))

# second mask, remove all unsuccessful requests (code != 200)
df2 = df2[df2['Status code'] == 200]
print("After unsuccessful requests removal", len(df2))

# Rows before: 51327
After images removal 5866
After unsuccessful requests removal 3288


After useless requests are removed, we are left with approximately 6.4% of the logs.

### 2.2 User Session Identification

This step is performed after data cleaning. The goal of user session identification is to divide the page access of each user into individual sessions. This step is non-trivial as it involves many things to take into consideration such as proxy servers, dynamic address, cases where multiple users access the same computer or one user uses multiple browsers and computers. These issues pose a major challenge in identification of sessions. However, there are some techniques that can provide additional information about session identification. The most common ones are cookies, dynamic web pages (with session ID in the URL) and user registrations.

One of the most popular techniques for identifying the session is using data from user logins. By combining user logins with the host and user agent it is possible to identify sessions. However it is not always possible to use this technique as not all users use their logins. So for identifying session, this practical uses the **time-out technique** described by Cooley (2002). The average session time-out used by most of the commercial products is 30 min. The session time-out is used in identifying the sessions. In the process of identifying sessions, each log entry is compared to identify if they belong to the same date. In the next step, if the date was same then, the IP address of the user is compared. If IP addresses are same, the time difference is calculated and compared with the average session time out to assign session id. The session id is incremented if the log values don't match. In the output, a session may include one or more requests merely depending on the session time out, IP and date.

### 2.3 Path Traversal

Path traversal provides an understanding of user access patterns in online environments. This analysis will not only help in improving the system design but also be able to lead to better marketing decisions. The users during their navigation to a Website visit some objects not due to their content but due to their location as they are part of navigation. This feature of traversal patterns increases the difficulty in extracting useful information from the traversal patterns of the users. In this practical, the date and time along with IP address is used to identify the path traversal of the users in their sessions. Once the sessions a recalculated, the requests made by the users in that sessions a reranked based on the time the request is made. The identification of the sessions and path traversal would help in performing associative mining for finding frequent sequential patterns, association and correlation among the sets of items. These associative mining rules are used to identify the correlation between the requests made during a session. These indicate the possible relationship between the pages that are often viewed even if they are not directly connected. Additionally, it can even reveal the association between users with particular interests.

### 2.4. User Identification

This is the most complex task in preprocessing web logs. The goal of this step is to identify unique users. But in most cases, the log files just provide the IP address of the users. In some cases the registered logins to web sites are found in the logs that aid in identifying the users. Even though user logins are not possible for every site, it is still possible to identify users by using IP address, browser and the rest of the fields. In this lab exercise, sessions are compared to identify the similarities in dates and IP address for identifying users. If the date and IP address are matched, it compares the time and browser. If the time difference is less than an hour and if browser is the same, it will imagine that same user is having two sessions. At any point of time, if the fields don’t match each other, the user id is incremented thinking that it is a different user.

Follow the code in cells below to execute all three tasks mentioned above. We provide comments on each line to ensure you understand each steps.

In [8]:
from collections import defaultdict
import datetime

# first, make a copy of df2 just in case
df3 = df2.copy()

# sort the rows based on datetime, descending
df3.sort_values(by='Datetime', inplace=True)

# initiate session ID and user ID to 0
session_id = 0
user_id = 0

# create a dictionaries to hold last access information
last_access = defaultdict(lambda:datetime.datetime.utcfromtimestamp(0))

# dictionary to find previous session, user ID and steps assigned to a specific date/ip/browser key
session_dict = defaultdict(lambda:1)
user_id_dict = defaultdict(lambda:1)
session_steps = defaultdict(lambda:1)

# function to be applied row wise
# for each row, produce session, user ID and path traversal
def get_log_user_info(row):
    # access global variables shared between all rows
    global session_id, user_id, session_dict, user_id_dict, session_steps, last_access
    
    session_key = str(row['Datetime'].date()) + '_' + row['Host']  # date + IP key for finding session
    user_key = str(row['Datetime'].date()) + '_' + row['Host'] + '_' + row['Agent']  # date + IP + browser key for finding user
    time_diff_session = row['Datetime'] - last_access[session_key]  # session time diff
    time_diff_user = row['Datetime'] - last_access[user_key]  # user time diff
    
    # if the time diff from previous session is > 30 mins, assign new session ID
    if time_diff_session.total_seconds() > 1800:
        session_id += 1
        session_dict[session_key] = session_id
    
    # if the time diff from previous session is > 60 mins, assign new user ID
    if time_diff_user.total_seconds() > 3600:
        user_id += 1
        user_id_dict[user_key] = user_id
        
    # update last access for session and user
    last_access[session_key] = row['Datetime']
    last_access[user_key] = row['Datetime']
    
    # assign extracted info from the row
    row['Session'] = session_dict[session_key]
    row['Step'] = session_steps[row['Session']]
    row['User_ID'] = user_id_dict[user_key]
    session_steps[row['Session']] += 1
    return row
    
# apply function above to get a new dataframe with added information
df3 = df3.apply(get_log_user_info, axis=1)

In [9]:
df3.head()

,Host,Identd,Authuser,Timezone,Request,Status code,Bytes Sent,Referrer,Agent,Method,Protocol,Datetime,Session,Step,User_ID
2,lj2559.inktomisearch.com,-,-,+1000],/code/Global/code/menu.html,200,6092,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,GET,HTTP/1.0,2005-04-18 21:16:55,1,1,1
3,c210-49-32-6.rochd2.qld.optusnet.com.au,-,-,+1000],/,200,7138,http://www.google.com.au/search?hl=en&q=snap+p...,Mozilla/5.0 (Macintosh; U; PPC Mac OS X; en-us...,GET,HTTP/1.1,2005-04-18 21:25:07,2,1,2
40,c210-49-32-6.rochd2.qld.optusnet.com.au,-,-,+1000],/services.html,200,15289,http://www.copyspecialists.com.au/,Mozilla/5.0 (Macintosh; U; PPC Mac OS X; en-us...,GET,HTTP/1.1,2005-04-18 21:25:16,2,2,2
84,c210-49-32-6.rochd2.qld.optusnet.com.au,-,-,+1000],/more.html,200,8975,http://www.copyspecialists.com.au/services.html,Mozilla/5.0 (Macintosh; U; PPC Mac OS X; en-us...,GET,HTTP/1.1,2005-04-18 21:25:39,2,3,2
99,c210-49-32-6.rochd2.qld.optusnet.com.au,-,-,+1000],/guarantee.html,200,5947,http://www.copyspecialists.com.au/more.html,Mozilla/5.0 (Macintosh; U; PPC Mac OS X; en-us...,GET,HTTP/1.1,2005-04-18 21:25:55,2,4,2


This output dataset now consists of IP address, requests, steps, session ids and user ids.

## 3. Applying Data Mining Methods

The pre-processed web log data provides a source dataset for data mining. Various data mining operations such as clustering and association mining can now be applied on this dataset. This application forms a task in the Assignment 2.